In [8]:
# KNN. 

In [15]:
from __future__ import division
from mnist import MNIST
from sklearn import tree
import numpy as np
from sklearn.metrics import classification_report,log_loss,accuracy_score,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from time import time
import pandas as pd
from datetime import datetime
from openpyxl import load_workbook
import re
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize


In [16]:
# Loading the data
mndata = MNIST('./')
images_train, labels_train = mndata.load_training()
images_test, labels_test = mndata.load_testing()
labels_test = np.array(labels_test)

images_train = np.array(images_train)
labels_train = np.array(labels_train)


In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
excel_file_name = 'KNN' + str(datetime.now().strftime('%Y%m%d%H%M%S')) + '.xlsx'


In [19]:
def classifaction_report_to_dataframe(report):
    report_data = []
    lines = report.split('\n')
    for line in lines[2:-5]:
        row = {}
        row_data = line.split('      ')
        if len(row_data)>4:
            row['class'] = row_data[1]
            row['precision'] = float(row_data[2])
            row['recall'] = float(row_data[3])
            row['f1_score'] = float(row_data[4])
            row['support'] = float(row_data[5])
            report_data.append(row)
    for line in lines[-4:-1]:
        row = {}
        p = re.compile("[a-z]+ [a-z]+")
        m = p.search(line)  
        row['class'] = m.group(0)
        p = re.compile("\d+.?\d+")
        row_data = p.findall(line)
        row['precision'] = float(row_data[0])
        row['recall'] = float(row_data[1])
        row['f1_score'] = float(row_data[2])
        row['support'] = float(row_data[3])
        report_data.append(row)
        
        
    dataframe = pd.DataFrame.from_dict(report_data)
    return dataframe


In [20]:
neighbours = [1,10,20,25,30,50,100,200,300,500]
df = pd.DataFrame(columns=['Number of neighbours', 'Accuracy','Normalized Accuracy', 
                 'Time_taken_train','Time_taken_test',
                 'Negative Log loss','Macro_auc','Micro_auc'])
for n in neighbours:
    neigh = KNeighborsClassifier(n_neighbors=n)
    
    start = time()
    neigh.fit(images_train, labels_train) 
    train_time = time() - start
    
    start = time()
    predictions = neigh.predict(images_test)
    test_time = time() - start
   
    accuracy = accuracy_score(predictions,labels_test,normalize=False)
    normalized_accuracy = accuracy_score(predictions,labels_test,normalize=True)
    log_loss_prediction = neigh.predict_proba(images_test)
    log_loss_val = log_loss(labels_test,log_loss_prediction) 
    classfication_repo = classification_report(labels_test, predictions)
    
    
    
    labels_train_one_hot=label_binarize(labels_test,classes=[0,1,2,3,4,5,6,7,8,9])
    predictions_one_hot=label_binarize(predictions,classes=[0,1,2,3,4,5,6,7,8,9])
    micro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="micro")
    macro_auc=roc_auc_score(labels_train_one_hot,predictions_one_hot,average="macro")

    print(micro_auc, macro_auc)

    
    
    
    df_data = [n,accuracy,normalized_accuracy,train_time,test_time,log_loss_val,macro_auc,micro_auc]
    df.loc[len(df)] = df_data
    
    

excel_writer = pd.ExcelWriter(excel_file_name,engine='xlsxwriter')
df.to_excel(excel_writer, sheet_name='knn')
df_classification_report = classifaction_report_to_dataframe(classfication_repo)
df_classification_report.to_excel(excel_writer, sheet_name='knn_cr')
excel_writer.save()    

0.9828333333333334 0.9826795366238563
0.9813888888888889 0.9812449298787067
0.9791666666666666 0.9790375266390281
0.9782777777777778 0.9781429743546859
0.9775555555555555 0.9774060582754938
0.9741111111111111 0.973923495391958
0.9688888888888888 0.9686384811224171
0.9605 0.9601203713817806
0.9543888888888888 0.9539039965122003
0.9466111111111112 0.9460124943114664
